In [1]:
import numpy as np

from human_terms_network import Human_Terms_Network
from dataset_load import *
from lr_baseline import LRBaseline

Using TensorFlow backend.


In [2]:
path = r"..\..\data\reviews_Amazon_Instant_Video_5.json.gz"

X, y = extract_review_amazon(path, 'reviewText')

y_label = y
# discard the review with 3. [1,2] = 0, [4,5] = 0
neutral_indices = np.where(y_label == 3)[0]
y_label[y_label<3] = 0
y_label[y_label>3] = 1

X_discarded = np.delete(X,neutral_indices)
y_discarded = np.delete(y_label, neutral_indices)

from sklearn.model_selection import train_test_split

X_train_split, X_test_split, y_train_amazon, y_test_amazon = train_test_split(
    X_discarded, y_discarded)

X_train_corpus_update = update_corpus_contraction(X_train_split)
X_test_corpus_update = update_corpus_contraction(X_test_split)



(75, 2)
corpus update start
corpus update end

(75, 2)
corpus update start
corpus update end



In [3]:
word_list, connotation = load_unigrams('./amazon-video-unigrams-more.txt', X_train_corpus_update, y_train_amazon)

In [4]:
# y_train_agreement, y_test_agreement = generate_appearance(X_train_corpus_update, X_test_corpus_update, 
#                                                           word_list, connotation)

In [5]:
av_baseline = LRBaseline(X_train_corpus_update, X_test_corpus_update, y_train_amazon, y_test_amazon, 
                           human_terms=word_list)

In [6]:
# (train, test)

print(av_baseline.baseline())
print(av_baseline.baseline(penalty='l1'))
print(av_baseline.human_terms_baseline())
print(av_baseline.human_terms_baseline(penalty='l1'))

(0.9539750647668394, 0.9243472981177899, 0.21583650560500464)
(0.9530440414507773, 0.926047358834244, 0.20783634000144277)
(0.9113503886010362, 0.9208911850177591, 0.22113137904816477, 0.24784456587735276)
(0.9125647668393783, 0.9223442040684533, 0.22126883816914306, 0.24784456587735276)


In [7]:
len(word_list)

78

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
clf = LogisticRegression(penalty='l1')
X_train = av_baseline.baseline_cv.transform(X_train_corpus_update)
X_test = av_baseline.baseline_cv.transform(X_test_corpus_update)

clf.fit(X_train, y_train_amazon)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
weight = clf.coef_.flatten()

In [11]:
word = av_baseline.baseline_cv.get_feature_names()

In [12]:
indices = np.argsort(weight)[::-1]

for i in indices[200:300]:
    print(word[i])

fourth
violence
complex
create
dynamic
pick
supernatural
enjoy
gritty
standard
miss
actresses
crime
ought
mixed
direction
afraid
light
biggest
documentary
week
networks
plan
still
seeing
cases
surprising
stars
prime
almost
budget
compared
places
since
teen
truly
currently
bosch
place
robert
keep
starting
satisfying
killing
each
element
top
explain
husband
spy
times
cable
intrigue
during
notice
process
apart
witty
peter
regular
government
end
plays
series
little
role
loving
plenty
everyone
detective
actions
happens
quite
period
using
you
forward
strange
clean
caught
number
company
sometimes
wish
working
although
dr
seasons
every
see
side
romantic
happen
age
experience
daughter
olyphant
help
aspects
strong


In [13]:
X_train = av_baseline.baseline_cv.transform(X_train_corpus_update)
X_test = av_baseline.baseline_cv.transform(X_test_corpus_update)
y_train = y_train_amazon
y_test = y_test_amazon

In [14]:
amazon_htm = Human_Terms_Network(input_shape=X_train.shape[1],
                               human_terms_shape=len(word_list),
                                domain='amazon', optimizer='adam')

In [15]:
amazon_htm.set_data(X_train, X_test, y_train_agreement, y_test_agreement, y_train, y_test)

NameError: name 'y_train_agreement' is not defined

In [ ]:
amazon_htm.train(epochs=10)

In [ ]:
amazon_htm.test(reject=True)

## has not been synchronized

In [ ]:
amazon_htm.test(reject=False)

In [ ]:
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])

indices = np.where(np.sum(concat_all, axis=1)!=0)
indices = indices[0]
len(indices)

